In [28]:
import gensim
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
import numpy as np
import pandas as pd
from collections import OrderedDict
from math import floor,ceil

function to load the files and do basic pre-processing

In [2]:
def loadAndProcess(filename): 
    f=open(filename)
    text=f.read()
    sents=sent_tokenize(text) #break the document into smaller sentences
    
    #remove sentences that have less than 5 words in total
    #i=0
    #while(i<len(sents)):
     #   sent=nltk.word_tokenize(sents[i])
      #  if len(sent)<5:
       #     del sents[i]
       # i=i+1
        
    #remove stop words
    for i in range(len(sents)):
        words=nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sents[i])
        words=[w.lower() for w in words if w not in stopwords.words('english')]
        words=" ".join(words)
        sents[i]=words
        
    return(sents)

function to load the files without any pre-processing, i.e. original sentences

In [5]:
def loadOnly(filename):
    f=open(filename)
    text=f.read()
    sents=sent_tokenize(text) #break the document into smaller sentences
    
    #remove sentences that have less than 5 words in total
    #i=0
    #while(i<len(sents)):
     #   sent=nltk.word_tokenize(sents[i])
      #  if len(sent)<5:
       #     del sents[i]
        #i=i+1
    return(sents)        

prepare a dictionary and corpus for lda

In [ ]:
def prepareCorpus(sentences):
    text=list()
    for i in range(len(sentences)):
        words=nltk.word_tokenize(sentences[i])
        text.append(words)
    dictionary = corpora.Dictionary(text)
    corpus = [dictionary.doc2bow(t) for t in text]
    return(dictionary,corpus)

load and run lda

In [7]:
original=loadOnly("data")  #original sentences without any pre processing
sents=loadAndProcess("data") #processed sentences
dic,corp=prepareCorpus(sents) #dictionary and corpus
numOfTopics=2           #change as per need
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corp, num_topics=numOfTopics, id2word = dic, passes=20)

create a sentences-topic matrix which is stored in a numpy array

In [11]:
docsTopicMatrix=np.zeros((len(sents),numOfTopics))
for i in range(len(sents)):
    topicsList=ldamodel.get_document_topics(corp[i],minimum_probability=0)
    for j in range(len(topicsList)):
        docsTopicMatrix[i,j]=topicsList[j][1]  

For each topic we pick out the best (2\*n/3k). This gives us a reduced set of sentences for each topic. These sentence numbers are stored in a numpy array.  

In [45]:
numOfSents=docsTopicMatrix.shape[0]
reducedNumOfSents=floor((2*numOfSents)/(3*numOfTopics))
reduced_sent_matrix=np.zeros((reducedNumOfSents,numOfTopics))

Loop through each column of the docs-Topic matrix and pick out the top 2n/3k sentence numbers for each topic. 

In [46]:
for j in range(numOfTopics):
    #for each column, store sentence number and weight pairs in a matrix
    temp_dict={}
    for i in range(numOfSents):
        temp_dict[i]=docsTopicMatrix[i,j]
    
    #choose the top 2n/3k sentences by finding the max element and deleting it from the dictionary 2n/3k times     
    for i in range(reducedNumOfSents):
        k=max(temp_dict, key=temp_dict.get)
        v=temp_dict.pop(k, None)
        reduced_sent_matrix[i,j]=k

In [47]:
reduced_sent_matrix

array([[ 92.,  60.],
       [  3.,  42.],
       [ 74.,  45.],
       [ 99.,   1.],
       [ 87.,  43.],
       [ 95.,  98.],
       [ 80.,  61.],
       [ 54.,  15.],
       [ 84.,  34.],
       [ 40.,  14.],
       [ 67.,  51.],
       [ 75.,  71.],
       [  8.,  16.],
       [ 10.,  90.],
       [ 35.,  38.],
       [ 66.,  65.],
       [ 23.,  62.],
       [ 86.,  41.],
       [ 78.,  46.],
       [ 27.,   7.],
       [ 59.,  58.],
       [ 70.,  96.],
       [ 69.,  32.],
       [ 97.,   5.],
       [ 93.,  83.],
       [ 25.,  56.],
       [ 36.,  55.],
       [ 12.,  63.],
       [ 31.,  44.],
       [ 91.,  89.],
       [  9.,  13.],
       [ 76.,  21.],
       [ 18.,  49.]])